In [12]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from MXband import MXband
from MPbandTool import MPbandTool
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [13]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [14]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8,14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [15]:
mb=MXband(phase,[80,180],straight_get,d,lower,upper,speedVar[0], speedVar[1],vol,px)
mb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125

Multi-objective solve log . . .

Index  Priority  Blend          Objective      Nodes  Time (sec.)  DetTime (ticks)
    1         0      3   1.1764016454e+03        440         0.06            32.57
status  = multi-objective optimal
time    = 0.079 s.
problem = MILP
gap     = 4.58476e+73%

object value: 2181


In [16]:
a=mb.get_dataframe()
a

,cross_number,distance,sg1,sg2,b1,b2,offset,p,t1,t2,z,u1,u2,w1,car_t1,w2,car_t2
0,1.0,800.0,0.282,0.380,0.282000,0.380,0.000000,0.0,91.520000,68.960000,80.000000,0.0,0.0,0.141000,11.139000,0.1900,37.570000
1,2.0,520.0,0.426,0.368,0.260000,0.368,0.000000,0.0,40.100000,46.300000,80.000000,0.0,0.0,0.285000,22.670000,0.1840,48.616000
2,3.0,500.0,0.260,0.230,0.260000,0.230,52.500000,0.0,62.500000,37.300000,80.000000,0.0,0.0,0.130000,62.770000,0.1150,82.385000
3,4.0,490.0,0.277,0.277,0.255000,0.230,35.000000,0.0,44.800000,35.000000,80.000000,0.0,0.0,0.130000,45.272500,0.1275,45.085000
4,5.0,370.0,0.255,0.255,0.255000,0.255,80.000000,0.0,26.475702,33.675702,80.000000,0.0,0.0,0.127500,90.072500,0.1275,90.072500
5,6.0,254.0,0.650,0.650,0.153785,0.650,55.802232,1.0,18.142857,18.142857,146.250000,0.0,0.0,0.076893,66.970875,0.3250,103.008482
6,7.0,585.0,0.243,0.243,0.243000,0.243,67.421250,0.0,73.125000,73.125000,146.250000,0.0,0.0,0.121500,85.069125,0.1215,85.069125
7,8.0,1020.0,0.165,0.165,0.165000,0.165,0.000000,1.0,117.435000,117.245000,80.000000,0.0,0.0,0.082500,6.517500,0.0825,6.517500
8,9.0,409.0,0.298,0.298,0.298000,0.165,32.115000,0.0,51.125000,34.195000,80.000000,0.0,0.0,0.149000,43.886000,0.2155,49.272500
9,10.0,547.0,0.379,0.379,0.379000,0.379,0.000000,1.0,45.387214,48.916338,89.344909,0.0,0.0,0.189500,16.741360,0.1895,16.741360


In [17]:
mpb=MPbandTool(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],a.p,a.offset/a.z,
            1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
mpb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 0.0000.
Tried aggregator 3 times.
MIP Presolve eliminated 767 rows and 250 columns.
MIP Presolve modified 923 coefficients.
Aggregator did 58 substitutions.
Reduced MIP has 454 rows, 236 columns, and 1164 nonzeros.
Reduced MIP has 36 binaries, 35 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.95 ticks)
Probing fixed 1 vars, tightened 57 bounds.
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 74 rows and 63 columns.
MIP Presolve modified 195 coefficients.
Reduced MIP has 380 rows, 173 columns, and 974 nonzeros.
Reduced MIP has 20 binaries, 33 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.57 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried ag

In [24]:
a=mpb.get_dataframe()
a

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u1,u2,u3,u4,u5,u6,bb1,bb2,tb1,tb2
0,1.0,800.0,0.282000,0.282000,0.000000,0.276343,0.000,0.380,0.000000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.282,0.380,72.142857,72.142857
1,2.0,520.0,0.260000,0.260000,0.000000,0.368000,0.000,0.368,0.000000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.260,0.368,37.142857,37.142857
2,3.0,500.0,0.260000,0.260000,0.000000,0.230000,0.000,0.230,52.500000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.260,0.230,50.714286,50.714286
3,4.0,490.0,0.255000,0.255000,0.000000,0.230000,0.000,0.230,35.000000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.255,0.230,35.000000,35.000000
4,5.0,370.0,0.255000,0.255000,0.000000,0.255000,0.000,0.255,80.000000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.255,0.255,41.428571,41.428571
5,6.0,254.0,0.153785,0.125785,0.153785,0.650000,0.000,0.650,55.802232,1.0,...,1.0,9.860000e-01,0.1275,1.0,1.000000e-08,1.000000e+00,0.243,0.650,18.142857,18.142857
6,7.0,585.0,0.243000,0.215000,0.243000,0.243000,0.000,0.243,67.421250,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.243,0.243,56.785714,56.785714
7,8.0,1020.0,0.165000,0.000000,0.165000,0.165000,0.000,0.000,0.000000,1.0,...,1.0,1.000000e-08,1.0000,1.0,1.000000e-08,1.000000e-08,0.165,0.165,87.857143,87.857143
8,9.0,409.0,0.100000,0.000000,0.172381,0.165000,0.000,0.000,32.115000,0.0,...,0.0,0.000000e+00,0.0000,0.0,0.000000e+00,0.000000e+00,0.298,0.165,44.214286,44.214286
9,10.0,547.0,0.379000,0.000000,0.379000,0.379000,0.379,0.000,0.000000,1.0,...,1.0,1.000000e-08,1.0000,1.0,7.845000e-01,1.000000e-08,0.379,0.379,54.071429,54.071429


In [19]:
from  SimAnalysis import SimAnalysis
df=mpb.get_dataframe()
df["dw1"]=np.ones_like(df.z)*dwt
df["dw2"]=np.ones_like(df.z)*dwt
sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","vissim","aa","zhongsan.inp"),
               os.path.join(os.getcwd(),"..","vissim","aa","vissim.ini"),
               os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.inp"))
sim.start()
spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.vlz'))
tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.rsz'))

In [28]:
spdx.to_csv("./sensitivity/spdx/MTband.csv",encoding="utf-8-sig")

In [26]:
i=8
delayx[["delay"+str(i),"stoped"+str(i),"stops"+str(i),"vehicle"+str(i)]]

,delay8,stoped8,stops8,vehicle8
0,0.0,0.0,0.00,0
1,343.6,207.8,10.00,1
2,498.3,353.5,14.40,5
3,527.5,406.7,18.50,10
4,605.7,467.2,28.40,5
5,710.0,521.2,41.08,13
6,599.1,445.7,27.74,34


In [27]:
a=tvltx["traveltime"+str(i)].astype(float)
a[a!=0].mean()

1306.75

In [25]:
a.b1.mean(),a.b2[:8].mean(),a.b3[4:].mean(),a.b4.mean(),a.b5[7:].mean(),a.b6[:8].mean(),a.bb1.mean(),a.bb2.mean()

(0.26482962335638993,
 0.20659813796123497,
 0.24479622847058138,
 0.309180197613002,
 0.23949999998509888,
 0.29449999995855614,
 0.28692307692307695,
 0.3171538461538462)

In [ ]:
dwm=np.array([[22.60083057,13.46496839,24.31,23.02457646,25.0084692,5.65743377,26.18040372,21.45,
                            21.30610731,15.91674887,10.69071851,15.91674887,21.15051345],
                           [27.90700128,24.76865382,26,33.1819511,33.15,10.60465754,31.59,21.45,30.94793897,
                            23.94176338,17.14793366,23.94176338,30.74566144]])

In [ ]:
np.array([26.3,11.7,27.6,28.3,29.4,24.7]).sum(),np.array([25.7,20.0,25.8,30.0,11.3,27.6]).sum(),np.array([24.6,11.7,29.9,27.7,29.4,25.6]).sum(),np.array([25.3,22.8,28.9,28.0,32.3,31.5]).sum()


In [ ]:
np.array([26.3,28.8]).mean(),np.array([27.6,30.0]).mean(),np.array([26.3,28.8]).mean(),np.array([25.3,30.5]).mean()
